<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [1]:
%%capture
!pip install category_encoders==2.*
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [2]:

data = './WA_Fn-UseC_-Telco-Customer-Churn+.csv'
df = pd.read_csv(data)
print(df.shape)


(7043, 21)


In [3]:
df['TotalCharges'] = df['TotalCharges'].replace(' ', np.nan)

In [4]:
df.isnull().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [5]:
# drop null values
df = df.dropna()

# convert TotalCharges to float
df['TotalCharges'] = df['TotalCharges'].astype('float')

In [6]:
X_features = df.drop(['Churn', 'customerID'], axis=1)
y_labels = df['Churn']


In [7]:
X_features.describe(exclude='number')

,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod
count,7032,7032,7032,7032,7032,7032,7032,7032,7032,7032,7032,7032,7032,7032,7032
unique,2,2,2,2,3,3,3,3,3,3,3,3,3,2,4
top,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check
freq,3549,3639,4933,6352,3385,3096,3497,3087,3094,3472,2809,2781,3875,4168,2365


In [8]:
def clean_df_to_numpy(dataframe):
    # create copy of dataframe
    df = dataframe.copy()
    
    # one hot encode and convert to numpy array
#     categorical_features = ['gender', 'SeniorCitizen', 'Partner',
#                         'Dependents', 'PhoneService', 'MultipleLines',
#                         'InternetService', 'OnlineSecurity', 'OnlineBackup',
#                         'DeviceProtection', 'TechSupport', 'StreamingTV',
#                         'StreamingMovies', 'Contract', 'PaperlessBilling',
#                         'PaymentMethod']
    
    encoder = ce.OneHotEncoder(use_cat_names=True)
    df = encoder.fit_transform(df)
    df_to_array = df.to_numpy()
    
    return df_to_array

In [9]:
# get the test train split
X_train, X_test, y_train, y_test = train_test_split(
    X_features, y_labels, test_size=0.20, random_state=42)

In [10]:
X_train = clean_df_to_numpy(X_train)
X_test = clean_df_to_numpy(X_test)

y_train = y_train.replace({'Yes': 1, 'No': 0})
y_test = y_test.replace({'Yes': 1, 'No': 0})

In [11]:
print(len(X_train) == len(y_train))
print(len(X_test) == len(y_test))

True
True


In [12]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(5625, 45)
(1407, 45)
(5625,)
(1407,)


In [13]:
# # Correct Encoding on Y
# num_classes = 2
# labels = keras.utils.to_categorical(telecom_labels, num_classes)


#### Baseline with average and baseline MLP Keras

In [14]:
# what is the baseline
df['Churn'].value_counts(normalize=True)

No     0.734215
Yes    0.265785
Name: Churn, dtype: float64

In [15]:
# num_classes = 2
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)

In [24]:
X_train.shape[1]
print(len(X_train))

5625


In [32]:
# Instantiate sequential model with layers

model = Sequential([
    
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')]
)

# complie the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# fit the model
fitted = model.fit(
    x=X_train, 
    y=y_train, 
    epochs=15, 
    validation_data=(X_test, y_test),
)

Epoch 1/15
176/176 [==============================] - 1s 4ms/step - loss: 12.6467 - accuracy: 0.6752 - val_loss: 4.1478 - val_accuracy: 0.3824
Epoch 2/15
176/176 [==============================] - 1s 3ms/step - loss: 8.8397 - accuracy: 0.6987 - val_loss: 5.5845 - val_accuracy: 0.7470
Epoch 3/15
176/176 [==============================] - 1s 3ms/step - loss: 4.5757 - accuracy: 0.7212 - val_loss: 3.1002 - val_accuracy: 0.3653
Epoch 4/15
176/176 [==============================] - 1s 4ms/step - loss: 3.4749 - accuracy: 0.7179 - val_loss: 0.8735 - val_accuracy: 0.7520
Epoch 5/15
176/176 [==============================] - 1s 4ms/step - loss: 2.6918 - accuracy: 0.7275 - val_loss: 1.7637 - val_accuracy: 0.7534
Epoch 6/15
176/176 [==============================] - 1s 3ms/step - loss: 1.1733 - accuracy: 0.7548 - val_loss: 0.6168 - val_accuracy: 0.7441
Epoch 7/15
176/176 [==============================] - 1s 3ms/step - loss: 1.1887 - accuracy: 0.7499 - val_loss: 0.6960 - val_accuracy: 0.7484
Epoch

## tuning batch size and epochs

In [34]:
# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential([
    
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')]
    )
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
param_grid = {'batch_size': [8, 16, 32, 64, 128],
              'epochs': [10, 15, 20, 25]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

Best: 0.7978666663169861 using {'batch_size': 8, 'epochs': 15}
Means: 0.7281777620315552, Stdev: 0.0668652417303076 with: {'batch_size': 8, 'epochs': 10}
Means: 0.7978666663169861, Stdev: 0.010793320718694606 with: {'batch_size': 8, 'epochs': 15}
Means: 0.7788444280624389, Stdev: 0.02155000370213255 with: {'batch_size': 8, 'epochs': 20}
Means: 0.7968000054359436, Stdev: 0.013766030185446008 with: {'batch_size': 8, 'epochs': 25}
Means: 0.7941333293914795, Stdev: 0.006424652935831748 with: {'batch_size': 16, 'epochs': 10}
Means: 0.785955548286438, Stdev: 0.012922857346024686 with: {'batch_size': 16, 'epochs': 15}
Means: 0.7475555539131165, Stdev: 0.07848037751971952 with: {'batch_size': 16, 'epochs': 20}
Means: 0.7354666709899902, Stdev: 0.11502456113252678 with: {'batch_size': 16, 'epochs': 25}
Means: 0.7624889016151428, Stdev: 0.04685851779827585 with: {'batch_size': 32, 'epochs': 10}
Means: 0.7343999981880188, Stdev: 0.08066764458758681 with: {'batch_size': 32, 'epochs': 15}
Means: 0.

In [ ]:
# keep the Batch_size at 8 and epochs at 15



## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?